In [118]:
import requests
import json
import pandas as pd

In [119]:
url = (
        r'https://www.jet2.com'
        r'/api/search/airportinformation/allairportinformation'
)
data = requests.get(url).json()

In [120]:
with open('../data/external/jet2_data.json') as f:
    data = json.load(f)

In [121]:
data = pd.DataFrame(data['Data'])

In [122]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   country               106 non-null    object
 1   label                 106 non-null    object
 2   code                  106 non-null    object
 3   isDepartureAirport    106 non-null    bool  
 4   isDestinationAirport  106 non-null    bool  
 5   destinationIataCodes  106 non-null    object
 6   searchTerms           106 non-null    object
 7   isEnabledForBooking   106 non-null    bool  
 8   airportUrlKey         106 non-null    object
dtypes: bool(3), object(6)
memory usage: 5.4+ KB


In [123]:
data.head(10)

,country,label,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,searchTerms,isEnabledForBooking,airportUrlKey
0,United Kingdom,Aberdeen,ABZ,True,False,,"[ABZ, Aberdeen, United Kingdom]",False,aberdeen
1,France,Albert (Picardie Somme),BYF,True,False,,"[BYF, Albert, (Picardie, Somme), France]",False,albert
2,Spain,Alicante ALC,ALC,True,True,BHX|NCL|MAN|STN|GLA|EDI|EMA|BFS|LBA,"[ALC, Alicante, ALC, Benidorm, Costa Blanca, S...",True,alicante
3,Spain,Almeria LEI,LEI,True,True,NCL|EMA|LBA|BHX|MAN|STN|GLA,"[LEI, Almeria, LEI, Spain]",True,almeria
4,Netherlands,Amsterdam AMS,AMS,True,True,LBA,"[AMS, Amsterdam, AMS, Holland, Netherlands, Ne...",True,amsterdam
5,Turkey,Antalya AYT,AYT,True,True,MAN|STN|BFS|NCL|GLA|EMA|EDI|BHX|LBA,"[AYT, Antalya, AYT, Turkey]",True,antalya
6,France,Avignon Provence,AVN,True,False,,"[AVN, Avignon, Provence, France]",False,avignon-provence
7,Spain,Barcelona Terminal 2 BCN,BCN,True,True,LBA|MAN|BHX|GLA,"[BCN, Barcelona, Terminal, BCN, ski, Andorra, ...",True,barcelona
8,United Kingdom,Belfast Intl BFS,BFS,True,True,LPA|ADB|PFO|TFS|NAP|PMI|ALC|AYT|REU|ZTH|CFU|MA...,"[BFS, Belfast, Intl, BFS, northern ireland, ir...",True,belfast
9,France,Bergerac EGC,EGC,True,True,BHX|LBA|MAN|STN,"[EGC, Bergerac, EGC, France]",True,bergerac


In [124]:
destinations = data[data['isDestinationAirport'] == True]

In [125]:
def string_splitter(string: str) -> list:
    return [
        code
        for code
        in string.split('|')
    ]

In [126]:
destinations['destinationIataCodes'] = destinations['destinationIataCodes'].apply(lambda x: string_splitter(x))

In [127]:
destinations = destinations.explode('destinationIataCodes')
destinations.drop(columns=['searchTerms', 'isEnabledForBooking', 'airportUrlKey'], inplace=True)
destinations.rename(
    columns={
        'label': 'Destination',
        'code': 'Destination Code',
        'destinationIataCodes': 'Departure Code'
    },
    inplace=True
)
destinations.head()

,country,Destination,Destination Code,isDepartureAirport,isDestinationAirport,Departure Code
2,Spain,Alicante ALC,ALC,True,True,BHX
2,Spain,Alicante ALC,ALC,True,True,NCL
2,Spain,Alicante ALC,ALC,True,True,MAN
2,Spain,Alicante ALC,ALC,True,True,STN
2,Spain,Alicante ALC,ALC,True,True,GLA
